In [2]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [5]:
data = pd.read_excel('../data/filtered_actuals.xlsx')

In [6]:
data

,FiscalYear,"Armstrong, Brooke","Breen, Daniel","Cosby, Russell","Curry, Ashley","Darrow, Terry","Eckert, Jeff","Esquivel, James","Forkner, Fiona","Haggar, James",...,"Selner, Bradley","Sheehy, Ahnie","Shipley, Christopher","Smith, Jubal","Stout, Christopher","Taguwa, Andrew","Toon, Larry","Weatherby, Samuel","Whitman, Paul","Wood, Alan"
0,2014,1380583.55,171398.16,770342.33,88839.39,1003383.83,750000.05,750000.01,384698.30,301113.55,...,3188933.63,151.72,356160.84,1771504.76,102575.75,901100.59,2259899.50,637791.99,1562838.05,856670.45
1,2015,631544.09,2194829.40,463416.90,335981.55,815760.82,997785.67,997785.62,998910.44,525300.89,...,2209823.93,142877.89,633904.48,7359378.40,250876.09,1011467.41,983978.26,1043269.72,1997295.13,786498.94
2,2016,1125976.16,1492879.42,1897362.90,401767.00,360931.29,995381.26,990905.52,1301178.85,641300.49,...,2076925.86,218536.00,808734.66,2113433.76,439799.56,714371.11,68855.76,576363.95,1207541.71,608604.08
3,2017,1526556.11,2425068.14,216952.35,267907.45,588227.17,812467.24,807640.25,665690.49,533198.91,...,2135517.42,333502.62,1325188.67,1495679.59,439281.72,998930.30,7801514.26,683077.75,363890.00,626262.49
4,2018,1786851.61,1557403.06,1035997.29,506123.66,429284.82,939797.56,939452.24,701267.31,540839.62,...,837927.59,394708.13,1221506.39,1477429.31,439691.67,1171395.37,543049.64,1662274.72,947842.79,433940.03
5,2019,2927638.63,1966813.19,2630950.86,393480.17,500000.00,610360.97,610355.66,589002.22,776827.65,...,3435213.05,500000.00,875584.42,9027557.98,837256.10,500000.00,563242.34,542493.41,798450.78,676387.34
6,2020,2774229.05,694513.79,85726.02,226290.45,69027.42,555576.34,555576.49,585002.05,765932.58,...,5020462.00,303459.66,1030727.50,1858116.40,729262.65,826222.42,204838.14,128476.25,1288304.98,92353.73
7,2021,1521039.35,1176575.40,1734397.73,529705.03,890021.64,1162108.21,1121160.16,334546.52,293815.62,...,3908492.32,217530.65,1382872.15,4769175.02,888623.37,650988.56,3920452.28,513034.18,1154146.70,273606.92
8,2022,138760.99,243170.51,262885.16,557611.48,240795.18,1101099.99,1071926.24,750000.00,1310808.71,...,4545074.40,488972.28,3319633.14,584342.95,900358.85,1277070.87,26433.64,559862.94,363262.31,500000.00
9,2023,-10737.07,427007.70,15142.76,592223.29,9041.67,986421.71,846948.54,1319272.27,2250000.00,...,2703434.77,126778.85,1530226.11,758521.60,1331616.06,1417720.02,26137.89,1128375.68,334568.10,180236.10


In [7]:
data_broker_target = pd.read_excel('../data/Actualandtarget_1401_10.xlsx')

In [8]:
broker_list = data.columns[1:]

In [10]:
data_broker_target_pivot = pd.pivot_table(data = data_broker_target, columns='FiscalYear', index='Broker', values='BrokerTarget').reset_index()

In [11]:
data_broker_target_pivot

FiscalYear,Broker,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Algrim, Phillip",700000.0,850000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Altman, Cribb",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000000.0
2,"Andrew, Megan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,755000.0,900000.0
3,"Angelle, Tiffany",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750000.0,3780000.0
4,"Ankenbrand, Kimarie",750000.0,500000.0,1600000.0,500000.0,750000.0,600000.0,600000.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
118,"Williams, Michael",NaN,NaN,NaN,NaN,NaN,NaN,NaN,300000.0,270000.0,335000.0
119,"Wood, Alan",600000.0,750000.0,800000.0,800000.0,2400000.0,NaN,800000.0,NaN,NaN,955000.0
120,"Wood, Jeff",NaN,NaN,NaN,NaN,NaN,NaN,NaN,500000.0,540000.0,670000.0
121,"Wright, Chris",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1600000.0,900000.0,1120000.0


In [12]:
data_broker_target_pivot_filtered = data_broker_target_pivot[data_broker_target_pivot['Broker'].isin(broker_list)].reset_index(drop=True)


In [13]:
data_broker_target_pivot_filtered_transposed = data_broker_target_pivot_filtered.set_index('Broker').T

In [14]:
data_broker_target_pivot_filtered_transposed

Broker,"Armstrong, Brooke","Breen, Daniel","Cosby, Russell","Curry, Ashley","Darrow, Terry","Eckert, Jeff","Esquivel, James","Forkner, Fiona","Haggar, James","Halstedt, Lauren",...,"Selner, Bradley","Sheehy, Ahnie","Shipley, Christopher","Smith, Jubal","Stout, Christopher","Taguwa, Andrew","Toon, Larry","Weatherby, Samuel","Whitman, Paul","Wood, Alan"
FiscalYear,,,,,,,,,,,,,,,,,,,,,
2014,1200000.0,500000.00,1000000.0,NaN,2100000.0,1400000.0,1400000.0,1300000.0,350000.0,NaN,...,1750000.0,NaN,400000.0,1200000.0,200000.0,750000.0,1500000.0,500000.0,1500000.0,600000.0
2015,1250000.0,500000.00,2000000.0,150000.0,1600000.0,1600000.0,1600000.0,1500000.0,1200000.0,125000.0,...,2500000.0,75000.0,400000.0,2500000.0,250000.0,855000.0,1500000.0,600000.0,1500000.0,750000.0
2016,1000000.0,1500000.00,1500000.0,400000.0,1400000.0,700000.0,700000.0,966667.0,600000.0,200000.0,...,2200000.0,160000.0,500000.0,3000000.0,500000.0,966667.0,1500000.0,1633334.0,1500000.0,800000.0
2017,750000.0,1379000.00,1000000.0,400000.0,400000.0,800000.0,800000.0,750000.0,1500000.0,300000.0,...,2000000.0,200000.0,750000.0,1500000.0,400000.0,750000.0,8650000.0,600000.0,1000000.0,800000.0
2018,1250000.0,2218666.67,1000000.0,400000.0,700000.0,2400000.0,2400000.0,2250000.0,1000000.0,NaN,...,1750000.0,200000.0,1800000.0,2218666.0,1200000.0,1000000.0,2000000.0,850000.0,3000000.0,2400000.0
2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020,2500000.0,NaN,2000000.0,NaN,NaN,NaN,NaN,1800000.0,NaN,NaN,...,2500000.0,NaN,NaN,NaN,NaN,2000000.0,600000.0,550000.0,800000.0,800000.0
2021,NaN,NaN,NaN,1500000.0,NaN,NaN,700000.0,NaN,NaN,200000.0,...,NaN,500000.0,2000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,NaN,1290000.00,NaN,1090000.0,315000.0,1630000.0,1630000.0,NaN,1250000.0,360000.0,...,NaN,900000.0,1265000.0,1540000.0,1260000.0,NaN,NaN,NaN,NaN,NaN


In [16]:
data_broker_target_pivot_filtered_transposed = data_broker_target_pivot_filtered_transposed.reset_index().rename(columns= {'index':'FiscalYear'})


In [4]:
data = data.reset_index().rename(columns= {'index':'FiscalYear'})


In [17]:
data_broker_target_pivot_filtered_transposed['FiscalYear'] = pd.to_datetime(data_broker_target_pivot_filtered_transposed['FiscalYear'], format = "%Y")

In [41]:
data['Time'] = pd.to_datetime(data['Time'], format = "%Y")

KeyError: 'Time'

In [ ]:
for name in data.columns[1:]:
    model = ARIMA(data[name], order=(2, 1, 3))
    model_fit = model.fit()
    data[name] = data[name].fillna(model_fit.predict(start=0, end=len(data)-1))


In [18]:
for name in data_broker_target_pivot_filtered_transposed.columns[1:]:
    model = ARIMA(data_broker_target_pivot_filtered_transposed[name], order=(2, 1, 3))
    model_fit = model.fit()
    data_broker_target_pivot_filtered_transposed[name] = data_broker_target_pivot_filtered_transposed[name].fillna(model_fit.predict(start=0, end=len(data_broker_target_pivot_filtered_transposed)-1))


/home/codespace/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


/home/codespace/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/codespace/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/codespace/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/codespace/.local/lib/python3.12/site-packages/statsmodels/base/model.py:607: Conve

In [19]:
data_broker_target_pivot_filtered_transposed.to_excel('../data/filtered_broker_target_filledna.xlsx')

In [20]:
data_broker_target_pivot_filtered_transposed

Broker,FiscalYear,"Armstrong, Brooke","Breen, Daniel","Cosby, Russell","Curry, Ashley","Darrow, Terry","Eckert, Jeff","Esquivel, James","Forkner, Fiona","Haggar, James",...,"Selner, Bradley","Sheehy, Ahnie","Shipley, Christopher","Smith, Jubal","Stout, Christopher","Taguwa, Andrew","Toon, Larry","Weatherby, Samuel","Whitman, Paul","Wood, Alan"
0,2014-01-01,1.200000e+06,5.000000e+05,1.000000e+06,0.000000e+00,2.100000e+06,1.400000e+06,1.400000e+06,1.300000e+06,3.500000e+05,...,1.750000e+06,0.000000e+00,4.000000e+05,1.200000e+06,2.000000e+05,7.500000e+05,1.500000e+06,5.000000e+05,1.500000e+06,6.000000e+05
1,2015-01-01,1.250000e+06,5.000000e+05,2.000000e+06,1.500000e+05,1.600000e+06,1.600000e+06,1.600000e+06,1.500000e+06,1.200000e+06,...,2.500000e+06,7.500000e+04,4.000000e+05,2.500000e+06,2.500000e+05,8.550000e+05,1.500000e+06,6.000000e+05,1.500000e+06,7.500000e+05
2,2016-01-01,1.000000e+06,1.500000e+06,1.500000e+06,4.000000e+05,1.400000e+06,7.000000e+05,7.000000e+05,9.666670e+05,6.000000e+05,...,2.200000e+06,1.600000e+05,5.000000e+05,3.000000e+06,5.000000e+05,9.666670e+05,1.500000e+06,1.633334e+06,1.500000e+06,8.000000e+05
3,2017-01-01,7.500000e+05,1.379000e+06,1.000000e+06,4.000000e+05,4.000000e+05,8.000000e+05,8.000000e+05,7.500000e+05,1.500000e+06,...,2.000000e+06,2.000000e+05,7.500000e+05,1.500000e+06,4.000000e+05,7.500000e+05,8.650000e+06,6.000000e+05,1.000000e+06,8.000000e+05
4,2018-01-01,1.250000e+06,2.218667e+06,1.000000e+06,4.000000e+05,7.000000e+05,2.400000e+06,2.400000e+06,2.250000e+06,1.000000e+06,...,1.750000e+06,2.000000e+05,1.800000e+06,2.218666e+06,1.200000e+06,1.000000e+06,2.000000e+06,8.500000e+05,3.000000e+06,2.400000e+06
5,2019-01-01,1.797766e+06,1.977076e+06,1.515785e+06,4.450111e+05,7.050786e+05,1.501899e+06,2.555989e+06,2.119291e+06,1.417596e+06,...,1.808278e+06,1.751064e+05,1.351294e+06,2.574303e+06,6.214291e+05,1.108331e+06,5.171692e+06,4.640914e+05,1.229895e+06,1.685388e+06
6,2020-01-01,2.500000e+06,2.347287e+06,2.000000e+06,3.514417e+05,8.218090e+05,7.387744e+05,1.489310e+06,1.800000e+06,1.232599e+06,...,2.500000e+06,1.438106e+05,1.642270e+06,2.405552e+06,1.397253e+06,2.000000e+06,6.000000e+05,5.500000e+05,8.000000e+05,8.000000e+05
7,2021-01-01,2.122976e+06,2.188235e+06,1.713554e+06,1.500000e+06,8.277283e+05,1.580360e+06,7.000000e+05,1.735445e+06,1.382997e+06,...,2.770005e+06,5.000000e+05,2.000000e+06,1.564799e+06,8.762097e+05,2.161284e+06,2.934364e+06,5.351797e+05,1.899507e+06,2.567850e+06
8,2022-01-01,1.633932e+06,1.290000e+06,1.241154e+06,1.090000e+06,3.150000e+05,1.630000e+06,1.630000e+06,2.168313e+06,1.250000e+06,...,2.728648e+06,9.000000e+05,1.265000e+06,1.540000e+06,1.260000e+06,2.010360e+06,2.911698e+06,5.808094e+05,1.636898e+06,1.696021e+06
9,2023-01-01,1.697376e+06,1.540000e+06,1.229080e+06,1.360000e+06,4.605749e+05,1.020000e+06,2.040000e+06,1.910000e+06,1.322577e+06,...,2.550000e+06,1.267266e+06,1.575000e+06,2.160000e+06,8.151310e+05,9.550000e+05,6.400000e+05,9.550000e+05,9.550000e+05,9.550000e+05


In [7]:
data['FiscalYear'] = pd.to_datetime(data['FiscalYear'], format="%Y")

In [9]:
data.to_excel('../data/filtered_filledna.xlsx',index=False)

In [43]:
import pandas as pd
from darts import TimeSeries
from darts.metrics.metrics import rmse
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel, NBEATSModel, RandomForest, NHiTSModel, NaiveMovingAverage, XGBModel, CatBoostModel
from darts.utils.utils import ModelMode

In [26]:
cutoff_year = '2021'

In [22]:
series = TimeSeries.from_dataframe(data, time_col="Time")

In [27]:
train_series, val_series = series.split_before(pd.Timestamp(cutoff_year))

In [46]:
def get_score(name, training_scaled, validation_scaled):

    score = {}

    # baseline 
    moving_average = NaiveMovingAverage(input_chunk_length=3)
    moving_average.fit(training_scaled[name])
    prediction = moving_average.predict(2024 - int(cutoff_year))

    moving_average_score = rmse(validation_scaled[name], prediction)
    score["MovingAverage"] = moving_average_score

    # statistical 
    es = ExponentialSmoothing(trend=ModelMode.ADDITIVE, seasonal=None)
    es.fit(training_scaled[name])
    prediction = es.predict(2024 - int(cutoff_year))


    es_score = rmse(validation_scaled[name], prediction)
    score["ExponentialSmoothing"] = es_score


    # ml
    rf = RandomForest(lags=1, n_estimators=10)
    rf.fit(training_scaled[name])
    prediction = rf.predict(2024 - int(cutoff_year))
    rf_score = rmse(validation_scaled[name], prediction)
    
    score["RandomForest"] = rf_score

    cb = CatBoostModel(lags=1, n_estimators=10)
    cb.fit(training_scaled[name])
    prediction = rf.predict(2024 - int(cutoff_year))
    cb_score = rmse(validation_scaled[name], prediction)
    
    score["CatBoost"] = cb_score

    xgb = XGBModel(lags=1, n_estimators=10)
    xgb.fit(training_scaled[name])
    prediction = xgb.predict(2024 - int(cutoff_year))
    xgb_score = rmse(validation_scaled[name], prediction)
    
    score["XGBModel"] = xgb_score

    # dl

    nhits = NHiTSModel(
    input_chunk_length=3,
    output_chunk_length=4,
    num_blocks=2,
    n_epochs=5,
)
    nhits.fit(training_scaled[name])
    prediction = nhits.predict(2024 - int(cutoff_year))
    nhits_score = rmse(validation_scaled[name], prediction)
    score["NHiTS"] = nhits_score


    return score
    



In [47]:
moving_average_list = []
exponential_smoothing_list = []
rf_list = []
nhits_list = []
xgb_list = []
catboost_list = []
for name in data.columns[1:]:

    scaler = Scaler()
    training_scaled = scaler.fit_transform(train_series[name])
    validation_scaled = scaler.transform(val_series[name])
    scores = get_score(name, training_scaled, validation_scaled)

    moving_average_list.append(scores['MovingAverage'])
    exponential_smoothing_list.append(scores['ExponentialSmoothing'])
    rf_list.append(scores['RandomForest'])
    nhits_list.append(scores['NHiTS'])
    xgb_list.append(scores['XGBModel'])
    catboost_list.append(scores['CatBoost'])
    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 1.6 M  | train
-------------------------------------------------------------
1.6 M     Trainable params
1.5 K     Non-trainable params
1.6 M     Total params
6.423     Total estimated model params size (MB)


Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 38.55it/s, train_loss=0.164]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 36.35it/s, train_loss=0.164]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 38.11it/s, train_loss=0.0773]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 35.98it/s, train_loss=0.0773]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 148.47it/s]


In [48]:
print(f"the aveage score of moving average is {round(sum(moving_average_list)/len(moving_average_list),3)}")
print(f"the aveage score of exponential Smoothing is {round(sum(exponential_smoothing_list)/len(exponential_smoothing_list),3)}")
print(f"the aveage score of random forest is {round(sum(rf_list)/len(rf_list),3)}")
print(f"the aveage score of nhits is {round(sum(nhits_list)/len(nhits_list),3)}")
print(f"the average score of catboost is {round(sum(catboost_list)/len(catboost_list),3)}")
print(f"the average score of xgboost is {round(sum(xgb_list)/len(xgb_list),3)}")



the aveage score of moving average is 0.512
the aveage score of exponential Smoothing is 0.651
the aveage score of random forest is 0.535
the aveage score of nhits is 0.779
the average score of catboost is 0.535
the average score of xgboost is 0.573
